Set up

In [ ]:
import psycopg2
import os
from se4g_helper import download_request, build_dataframe, update_dataset, update_dashboard_dataset

In [ ]:
# Set the working directory
current_dir = os.getcwd()
print('current_dir: ',current_dir)
parent_dir = os.path.dirname(current_dir)

os.chdir(parent_dir)

print('Current working directory:', os.getcwd())

In [ ]:
# Download and get the dataframe file name
dir = download_request(folder_out = 'data_prova')

# Build the dataframe with the required structure
new_df = build_dataframe(dir, folder_out = 'data_prova')

# Update the final dataset
update_dataset(new_df, folder_out = 'data_prova')
# Update the dashboard dataset 
update_dashboard_dataset(new_df,folder_out = 'data_prova')

In [ ]:
def update(new_data,conn,df_columns = ['station_code', 
							      'station_name', 
								  'station_altitude', 
								  'network_countrycode', 
								  'pollutant', 
								  'value_datetime_begin',
								  'value_datetime_end',
								  'value_datetime_updated',
								  'value_numeric']):

    cur = conn.cursor()
    cur.execute(f"INSERT INTO se4g ({df_columns}) VALUES (%s, %s, %.1f, %s, %s, %s, %s, %s, %f)", new_data)


In [ ]:
ip = '192.168.30.19'

conn = psycopg2.connect(
    host = 'localhost',
    database = "se4g",
    user = "postgres",
    password = "carIs3198")

update(new_df,conn)

#Save changes using the commit() function
conn.commit()
#Close the database connection
conn.close()


In [ ]:
cur.execute('''UPDATE pollutant_detection
 (station_code TEXT , 
 name_pollutant TEXT, 
 data_time_begin DATE, 
 data_time_end DATE,
 data_time_update DATE,
 value_pollutant DECIMAL,
 PRIMARY KEY (name_pollutant, station_code, data_time_begin,data_time_end ))''')